In [4]:
from gurobipy import Model, GRB, quicksum
import numpy as np
from itertools import product

In [5]:
from itertools import product

num_weeks = 9
num_consmatches_1 = 4
num_consmatches_2 = 5

def valid_pattern(pattern):
    h_count = pattern.count('H')
    a_count = pattern.count('A')
    
    # Check the total H/A counts
    if not (
        (h_count == num_consmatches_1 and a_count == num_consmatches_2) or
        (h_count == num_consmatches_2 and a_count == num_consmatches_1)
    ):
        return False
    
    # Check for "breaks" only between odd/even round pairs
    for i in range(0, len(pattern), 2):
        if i + 1 < len(pattern):
            if pattern[i] == pattern[i + 1]:
                return False
    
    return True

all_patterns = product('HA', repeat=num_weeks)
valid_patterns = [
    ''.join(p) for p in all_patterns
    if valid_pattern(p)
]

num_patterns = len(valid_patterns)
print(f"Total valid patterns: {len(valid_patterns)}")
print(valid_patterns)


Total valid patterns: 32
['HAHAHAHAH', 'HAHAHAHAA', 'HAHAHAAHH', 'HAHAHAAHA', 'HAHAAHHAH', 'HAHAAHHAA', 'HAHAAHAHH', 'HAHAAHAHA', 'HAAHHAHAH', 'HAAHHAHAA', 'HAAHHAAHH', 'HAAHHAAHA', 'HAAHAHHAH', 'HAAHAHHAA', 'HAAHAHAHH', 'HAAHAHAHA', 'AHHAHAHAH', 'AHHAHAHAA', 'AHHAHAAHH', 'AHHAHAAHA', 'AHHAAHHAH', 'AHHAAHHAA', 'AHHAAHAHH', 'AHHAAHAHA', 'AHAHHAHAH', 'AHAHHAHAA', 'AHAHHAAHH', 'AHAHHAAHA', 'AHAHAHHAH', 'AHAHAHHAA', 'AHAHAHAHH', 'AHAHAHAHA']


In [6]:
# Dictionary `feasible_opponents` where feasible_opponents[i, t]
# is the set of feasible opponents for pattern i in week t


# Initialize the feasible opponents dictionary
feasible_opponents = {}

# Iterate through each pattern and week
num_patterns = len(valid_patterns)
num_weeks = len(valid_patterns[0])

for i in range(num_patterns):
    for t in range(num_weeks):
        # Get the H/A status of pattern i in week t
        current_status = valid_patterns[i][t]
        
        # Find all patterns that have the opposite status in week t
        feasible_opponents[i, t] = {j for j in range(num_patterns) if j != i and valid_patterns[j][t] != current_status}


# Print the feasible opponents dictionary for verification
for key, value in feasible_opponents.items():
    print(f"Pattern {key[0]} in week {key[1]} can play against patterns {value}")


Pattern 0 in week 0 can play against patterns {16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Pattern 0 in week 1 can play against patterns {16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Pattern 0 in week 2 can play against patterns {8, 9, 10, 11, 12, 13, 14, 15, 24, 25, 26, 27, 28, 29, 30, 31}
Pattern 0 in week 3 can play against patterns {8, 9, 10, 11, 12, 13, 14, 15, 24, 25, 26, 27, 28, 29, 30, 31}
Pattern 0 in week 4 can play against patterns {4, 5, 6, 7, 12, 13, 14, 15, 20, 21, 22, 23, 28, 29, 30, 31}
Pattern 0 in week 5 can play against patterns {4, 5, 6, 7, 12, 13, 14, 15, 20, 21, 22, 23, 28, 29, 30, 31}
Pattern 0 in week 6 can play against patterns {2, 3, 6, 7, 10, 11, 14, 15, 18, 19, 22, 23, 26, 27, 30, 31}
Pattern 0 in week 7 can play against patterns {2, 3, 6, 7, 10, 11, 14, 15, 18, 19, 22, 23, 26, 27, 30, 31}
Pattern 0 in week 8 can play against patterns {1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31}
Pattern 1 in week 0 can pl

In [7]:

from gurobipy import Model, GRB, quicksum

# Initialize m
m = Model("Sports_Scheduling")

# Parameters
patterns = range(len(valid_patterns))
weeks = range(num_weeks - 1)
n_teams = 10

# Binary decision variable: p[i, j, t] is 1 if pattern i is paired with pattern j in week t
p = m.addVars(patterns, patterns, weeks, vtype=GRB.BINARY, name="p")

# New binary decision variable y[i]: 1 if pattern i is selected as part of the 10 patterns, 0 otherwise
y = m.addVars(patterns, vtype=GRB.BINARY, name="y")

# Constraint: Select exactly 10 patterns to be used consistently across all weeks
m.addConstr(quicksum(y[i] for i in patterns) == n_teams, name="select_10_patterns")

# Constraint: Ensure each pattern used in a week is one of the 10 selected patterns
for i in patterns:
    for t in weeks:
        if (i,t) in feasible_opponents:
            m.addConstr(quicksum(p[i, j, t] for j in feasible_opponents[i,t] if i<j) + quicksum(p[j, i, t] for j in feasible_opponents[i,t] if j<i)  == y[i], name=f"pattern_consistency_{i}_{t}")

for j in patterns:
    if i>=j:
        for k in weeks:
            p[i,j,k].ub = 0

for i in patterns:
    for k in weeks:
        for j in patterns:
            if i<j:
                if j not in feasible_opponents[i,k]:
                    p[i,j,k].ub = 0

# Constraint: Each pair of selected patterns is selected for exactly one week
for i in patterns:
    for j in patterns:
            if i != j:
                m.addConstr(quicksum(p[i, j, t] for t in weeks) <= y[i], name=f"B3_unique_match_{i}_{j}")



Set parameter Username
Academic license - for non-commercial use only - expires 2025-11-04


In [8]:

m.optimize()
if m.status == GRB.OPTIMAL:
    patterns_opt = {(i,j,k): p[i,j,k].x for i in patterns for j in patterns for k in weeks if p[i,j,k].x > 0.5 }
    selected_patterns = {j: y[j].x for j in patterns if y[j].x > 0.5}

    solution_found = {
        "p": patterns_opt,
        "y": selected_patterns
    }

else:
    print("No optimal solution found")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1249 rows, 8224 columns and 13312 nonzeros
Model fingerprint: 0x019478be
Variable types: 0 continuous, 8224 integer (8224 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Presolve removed 512 rows and 6144 columns
Presolve time: 0.02s
Presolved: 737 rows, 2080 columns, 6912 nonzeros
Variable types: 0 continuous, 2080 integer (2080 binary)

Root relaxation: objective 0.000000e+00, 701 iterations, 0.02 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   9

In [9]:
import pandas as pd

# Extract weeks dynamically
weeks = sorted({t for _, _, t in patterns_opt.keys()})
calendar_data = {f"Week {t+1}": [] for t in weeks}

# Populate calendar data for each week 
for t in weeks:
    week_pairs = []
    for (i, j, w) in patterns_opt:
        if w == t:
            week_pairs.append(f"Pattern {i} vs Pattern {j}")
    calendar_data[f"Week {t+1}"] = week_pairs

# Create and display the DataFrame
calendar_df = pd.DataFrame(calendar_data)
calendar_df

,Week 1,Week 2,Week 3,Week 4,Week 5,Week 6,Week 7,Week 8
0,Pattern 2 vs Pattern 29,Pattern 2 vs Pattern 31,Pattern 2 vs Pattern 25,Pattern 2 vs Pattern 27,Pattern 2 vs Pattern 4,Pattern 2 vs Pattern 6,Pattern 2 vs Pattern 13,Pattern 2 vs Pattern 16
1,Pattern 3 vs Pattern 16,Pattern 3 vs Pattern 27,Pattern 3 vs Pattern 31,Pattern 3 vs Pattern 25,Pattern 3 vs Pattern 6,Pattern 3 vs Pattern 29,Pattern 3 vs Pattern 4,Pattern 3 vs Pattern 13
2,Pattern 4 vs Pattern 27,Pattern 4 vs Pattern 16,Pattern 4 vs Pattern 29,Pattern 4 vs Pattern 13,Pattern 13 vs Pattern 16,Pattern 4 vs Pattern 25,Pattern 6 vs Pattern 16,Pattern 4 vs Pattern 6
3,Pattern 6 vs Pattern 25,Pattern 6 vs Pattern 29,Pattern 6 vs Pattern 13,Pattern 6 vs Pattern 31,Pattern 25 vs Pattern 29,Pattern 13 vs Pattern 27,Pattern 25 vs Pattern 31,Pattern 25 vs Pattern 27
4,Pattern 13 vs Pattern 31,Pattern 13 vs Pattern 25,Pattern 16 vs Pattern 27,Pattern 16 vs Pattern 29,Pattern 27 vs Pattern 31,Pattern 16 vs Pattern 31,Pattern 27 vs Pattern 29,Pattern 29 vs Pattern 31


In [10]:
solution_found

{'p': {(2, 4, 4): 1.0,
  (2, 6, 5): 1.0,
  (2, 13, 6): 1.0,
  (2, 16, 7): 1.0,
  (2, 25, 2): 1.0,
  (2, 27, 3): 1.0,
  (2, 29, 0): 1.0,
  (2, 31, 1): 1.0,
  (3, 4, 6): 1.0,
  (3, 6, 4): 1.0,
  (3, 13, 7): 1.0,
  (3, 16, 0): 1.0,
  (3, 25, 3): 1.0,
  (3, 27, 1): 1.0,
  (3, 29, 5): 1.0,
  (3, 31, 2): 1.0,
  (4, 6, 7): 1.0,
  (4, 13, 3): 1.0,
  (4, 16, 1): 1.0,
  (4, 25, 5): 1.0,
  (4, 27, 0): 1.0,
  (4, 29, 2): 1.0,
  (6, 13, 2): 1.0,
  (6, 16, 6): 1.0,
  (6, 25, 0): 1.0,
  (6, 29, 1): 1.0,
  (6, 31, 3): 1.0,
  (13, 16, 4): 1.0,
  (13, 25, 1): 1.0,
  (13, 27, 5): 1.0,
  (13, 31, 0): 1.0,
  (16, 27, 2): 1.0,
  (16, 29, 3): 1.0,
  (16, 31, 5): 1.0,
  (25, 27, 7): 1.0,
  (25, 29, 4): 1.0,
  (25, 31, 6): 1.0,
  (27, 29, 6): 1.0,
  (27, 31, 4): 1.0,
  (29, 31, 7): 1.0},
 'y': {2: 1.0,
  3: 1.0,
  4: 1.0,
  6: 1.0,
  13: 1.0,
  16: 1.0,
  25: 1.0,
  27: 1.0,
  29: 1.0,
  31: 1.0}}